In [10]:
import types
import json
import os
import csv
import requests


def exhaustively_write(data_struc, csv_file):
   if type(data_struc) is types.DictType:
      for key in data_struc.keys():
         exhaustively_write(data_struc[key],csv_file)
   elif type(data_struc) is types.ListType:
      for i in xrange(0,len(data_struc)):
         exhaustively_write(data_struc[i],csv_file)
   else:
      csv_file.write(str(data_struc)+',')


for f in os.listdir('mini_dump'):
   if f.find('json') != -1:

        json_data = open('mini_dump/'+f)
        data = json.load(json_data)
        if len(data['participants']) == 10:
            csv_file = open('parsed/'+f.split('.json')[0]+'.csv','w+')

            for key in data.keys():
               if key == 'participants':
                  for i in range(1,10):
                     exhaustively_write(data['participants'][i],csv_file)
                     csv_file.write('\n')
               else:
                  exhaustively_write(data[key],csv_file)
                  csv_file.write('\n')


In [6]:
import json
import os

def insertComma(fileName):
   fileName.write(' , ')

def writetoFile(fileName,data):
   fileName.write(str(data))
   insertComma(fileName)

DUMP_DIR = 'mini_dump/'
counter = 0
trainingData = open('parsig_practice.csv','w+')
smallConst = 1e-15   #Don't divide by zero

trainingData.write('matchID, participant, Win, FirstBlood, firstTower, firstTowerAssist, Kills, Assists, Deaths, GoldEarned, GoldSpent, TotalDamageDealt, MagicDamageDealt, PhysicalDamageDealt, TotalDamageTaken, MinionsKilled, CrowdControl, WardsPlaced\n')

for f in os.listdir(DUMP_DIR):
   if f.find('json') != -1:

      json_data = open(DUMP_DIR + f)
      data = json.load(json_data)
      json_data.close()

      if len(data['participants']) == 10:  #verify valid game with all players
         counter += 1

         teamKills = [smallConst,smallConst]
         teamAssists = [smallConst,smallConst]
         teamDeaths = [smallConst,smallConst]
         teamGoldEarned = [smallConst, smallConst]
         teamGoldSpent = [smallConst, smallConst]
         teamDamageDealt = [smallConst, smallConst]
         teamMagicDamageDealt = [smallConst, smallConst]
         teamPhysicalDamageDealt = [smallConst, smallConst]
         teamTotalDamageTaken = [smallConst, smallConst]
         teamMinionsKilled = [smallConst, smallConst]
         teamCrowdControl= [smallConst, smallConst]
         teamWards = [smallConst, smallConst]

         for i in range(0,10): # summing loop for normalization
            stats = data['participants'][i]['stats']
            if data['participants'][i]['teamId'] == data['teams'][0]['teamId']:  #team 0
               teamKills[0] += stats['kills']
               teamAssists[0] += stats['assists']
               teamDeaths[0] += stats['deaths']
               teamGoldEarned[0] += stats['goldEarned']
               teamGoldSpent[0] += stats['goldSpent']
               teamDamageDealt[0] += stats['totalDamageDealt']
               teamMagicDamageDealt[0] += stats['magicDamageDealt']
               teamPhysicalDamageDealt[0] += stats['physicalDamageDealt']
               teamTotalDamageTaken[0] += stats['totalDamageTaken']
               teamMinionsKilled[0] += stats['minionsKilled']
               teamCrowdControl[0] += stats['totalTimeCrowdControlDealt']
               teamWards[0] += stats['wardsPlaced']

            else: # team 1
               teamKills[1] += stats['kills']
               teamAssists[1] += stats['assists']
               teamDeaths[1] += stats['deaths']
               teamGoldEarned[1] += stats['goldEarned']
               teamGoldSpent[1] += stats['goldSpent']
               teamDamageDealt[1] += stats['totalDamageDealt']
               teamMagicDamageDealt[1] += stats['magicDamageDealt']
               teamPhysicalDamageDealt[1] += stats['physicalDamageDealt']
               teamTotalDamageTaken[1] += stats['totalDamageTaken']
               teamMinionsKilled[1] += stats['minionsKilled']
               teamCrowdControl[1] += stats['totalTimeCrowdControlDealt']
               teamWards[1] += stats['wardsPlaced']

         for i in range(0,10):

            teamID = 0 if data['participants'][i]['teamId'] == data['teams'][0]['teamId'] else 1
            stats = data['participants'][i]['stats']

            # General Info -- not used for clustering
            writetoFile(trainingData,data['matchId'])
            writetoFile(trainingData,data['participantIdentities'][i]['participantId'])

            # Booleans -- NOT NORMALIZED
            writetoFile(trainingData,int(data['teams'][teamID]['winner']))
            if stats.has_key('firstBloodKill'):
               writetoFile(trainingData,int(stats['firstBloodKill']))
            else:
               writetoFile(trainingData, 0)

            writetoFile(trainingData,int(stats['firstTowerKill']))
            writetoFile(trainingData,int(stats['firstTowerAssist']))

            writetoFile(trainingData, stats['kills']/teamKills[teamID])
            writetoFile(trainingData, stats['assists']/teamAssists[teamID])
            writetoFile(trainingData, stats['deaths']/teamDeaths[teamID])
            writetoFile(trainingData, stats['goldEarned']/teamGoldEarned[teamID])
            writetoFile(trainingData, stats['goldSpent']/teamGoldSpent[teamID])
            writetoFile(trainingData, stats['totalDamageDealt']/teamDamageDealt[teamID])
            writetoFile(trainingData, stats['magicDamageDealt']/teamMagicDamageDealt[teamID])
            writetoFile(trainingData, stats['physicalDamageDealt']/teamMagicDamageDealt[teamID])
            writetoFile(trainingData, stats['totalDamageTaken']/teamTotalDamageTaken[teamID])
            writetoFile(trainingData, stats['minionsKilled']/teamMinionsKilled[teamID])
            writetoFile(trainingData, stats['totalTimeCrowdControlDealt']/teamCrowdControl[teamID])
            writetoFile(trainingData, stats['wardsPlaced']/teamWards[teamID])

            trainingData.write('\n')
            #print counter

In [11]:
########## Champion ID to Champion Name converter ##########

# GET the static data on champion keys and names
static_champ_data = requests.get("https://global.api.pvp.net/api/lol/static-data/na/v1.2/champion?locale=en_US&api_key=b00c992b-6fc1-4795-abcc-7db85c1b72fa").json()
champs = static_champ_data['data']
champ_names = static_champ_data['data'].keys()

# initializing index and empty dictionaries
i = 0
list_champs = []
list_champ_ids = []

# filling in the lists
for i in range(len(champ_names)):
    this_champ = champs[champ_names[i]]
    list_champs.append(this_champ['key'])
    list_champ_ids.append(this_champ['id'])
    
# champion ID and name dictionary    
champ_dict = dict(zip(list_champ_ids, list_champs))

# Wukong's name is not correct (MonkeyKing)
champ_dict[62] = 'Wukong'

# function that converts a champion ID to champion name
def champion_name(champion_id):
    if champion_id in champ_dict:
        return champ_dict[champion_id]
    else:
        return 'no_name'

In [20]:
import json
import os

def insertComma(fileName):
   fileName.write(' , ')

def writetoFile(fileName,data):
   fileName.write(str(data))
   insertComma(fileName)

DUMP_DIR = 'mini_dump/'
counter = 0
trainingData = open('unnormarlized_data.csv','w+')
smallConst = 1e-15   #Don't divide by zero

trainingData.write('matchID, participant, champ, Win, firstTowerAssist, Kills, Assists, Deaths, GoldEarned, GoldSpent, TotalDamageDealt, MagicDamageDealt, PhysicalDamageDealt, TotalDamageTaken, MinionsKilled, CrowdControl, WardsPlaced\n')

for f in os.listdir(DUMP_DIR):
   if f.find('json') != -1:

      json_data = open(DUMP_DIR + f)
      data = json.load(json_data)
      json_data.close()

      if len(data['participants']) == 10:  #verify valid game with all players
         counter += 1

         teamKills = [smallConst,smallConst]
         teamAssists = [smallConst,smallConst]
         teamDeaths = [smallConst,smallConst]
         teamGoldEarned = [smallConst, smallConst]
         teamGoldSpent = [smallConst, smallConst]
         teamDamageDealt = [smallConst, smallConst]
         teamMagicDamageDealt = [smallConst, smallConst]
         teamPhysicalDamageDealt = [smallConst, smallConst]
         teamTotalDamageTaken = [smallConst, smallConst]
         teamMinionsKilled = [smallConst, smallConst]
         teamCrowdControl= [smallConst, smallConst]
         teamWards = [smallConst, smallConst]

         for i in range(0,10):

            teamID = 0 if data['participants'][i]['teamId'] == data['teams'][0]['teamId'] else 1
            stats = data['participants'][i]['stats']

            # General Info -- not used for clustering
            writetoFile(trainingData, data['matchId'])
            writetoFile(trainingData, data['participantIdentities'][i]['participantId'])
            writetoFile(trainingData, champ_dict[data['participants'][i]['championId']])

            # Booleans -- NOT NORMALIZED
            writetoFile(trainingData,int(data['teams'][teamID]['winner']))
            if stats.has_key('firstBloodKill'):
               writetoFile(trainingData,int(stats['firstBloodKill']))
            else:
               writetoFile(trainingData, 0)

            writetoFile(trainingData,int(stats['firstTowerKill']))
            writetoFile(trainingData,int(stats['firstTowerAssist']))

            writetoFile(trainingData, stats['kills'])
            writetoFile(trainingData, stats['assists'])
            writetoFile(trainingData, stats['deaths'])
            writetoFile(trainingData, stats['goldEarned'])
            writetoFile(trainingData, stats['goldSpent'])
            writetoFile(trainingData, stats['totalDamageDealt'])
            writetoFile(trainingData, stats['magicDamageDealt'])
            writetoFile(trainingData, stats['physicalDamageDealt'])
            writetoFile(trainingData, stats['totalDamageTaken'])
            writetoFile(trainingData, stats['minionsKilled'])
            writetoFile(trainingData, stats['totalTimeCrowdControlDealt'])
            writetoFile(trainingData, stats['wardsPlaced'])

            trainingData.write('\n')
            print counter

1
1
1
1
1
1
1
1
1
1
2
2
2
2
2
2
2
2
2
2
3
3
3
3
3
3
3
3
3
3
4
4
4
4
4
4
4
4
4
4
5
5
5
5
5
5
5
5
5
5
6
6
6
6
6
6
6
6
6
6
7
7
7
7
7
7
7
7
7
7
8
8
8
8
8
8
8
8
8
8
9
9
9
9
9
9
9
9
9
9
10
10
10
10
10
10
10
10
10
10
11
11
11
11
11
11
11
11
11
11
12
12
12
12
12
12
12
12
12
12
13
13
13
13
13
13
13
13
13
13
14
14
14
14
14
14
14
14
14
14
15
15
15
15
15
15
15
15
15
15
16
16
16
16
16
16
16
16
16
16


In [18]:
os.getcwd()

'/Users/seungjaecha/cs109/Project/CS109_Final'